# Segmenting and Clustering Neighbourhoods in Toronto, Part 3

In [1]:
!pip install geocoder
!pip install geopy
!pip install folium
print("installed")

installed


In [2]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import geocoder

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("imported")

imported


#### below is stuff from part 1 and 2, scroll down further to find the part 3

### Web Scraping

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup =BeautifulSoup(source.text,'lxml')

In [4]:
data = []
table = soup.find(class_='wikitable')
index=0
for tr in table.find_all('tr'):
    index += 1
    if index>1:
        section = [] 
        for td in tr.find_all('td'):
            section.append(td.text.rstrip())
        data.append(section)

In [5]:
df = pd.DataFrame(data = data,columns=['Postcode','Borough','Neighbourhood'])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning

In [6]:
#drop the rows that have 'not assigned' in the borough column

df = df[df.Borough!='Not assigned']

In [7]:
df['Neighbourhood'].replace("not assigned",df['Borough'],inplace=True)

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df.shape

(103, 3)

In [10]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70909000000006, -79.36409999999995]

In [11]:
# retrieving postal code co-ordinates

postal_codes = df['Postcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [12]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
df[df.Postcode == 'M5G']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
40,M5G,Downtown Toronto,Central Bay Street,43.73903,-79.46732


In [14]:
df.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.65514,-79.36265
3,M4A,North York,Victoria Village,43.72321,-79.45141
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66449,-79.39302
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66277,-79.52831
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81153,-79.19552
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.70794,-79.31160
9,M1B,Scarborough,"Malvern, Rouge",43.65736,-79.37818
11,M3B,North York,Don Mills,43.65279,-79.55406
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.78564,-79.15871
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.72184,-79.34340


In [15]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66449,-79.39302
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81153,-79.19552
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.72184,-79.34340
3,M5C,Downtown Toronto,St. James Town,43.76820,-79.21761
4,M4E,East Toronto,The Beaches,43.64970,-79.38258
5,M5E,Downtown Toronto,Berczy Park,43.66509,-79.43871
6,M5G,Downtown Toronto,Central Bay Street,43.73903,-79.46732
7,M6G,Downtown Toronto,Christie,43.68375,-79.35528
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.71381,-79.48831
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.75948,-79.55725


In [16]:
def fill_lat(address,j):
    add=address
    geolocator =  Nominatim(user_agent='Toronto_agent')
    location = geolocator.geocode(add)
    toronto_df['Latitude'].iloc[j] = location.latitude
    toronto_df['Longitude'].iloc[j] = location.longitude
    #print('Latitude of {} is {}'.format(address,toronto_df['Latitude']))
    #print('Longitude of {} is {}'.format(address, toronto_df['Longitude']))

      
for i,lat in enumerate(toronto_df.Latitude):
    j=i+21
    if(df.Latitude.isnull().any() and j<38):
        address=toronto_df['Neighbourhood'].str.rsplit(",")[j][0]
        print(j,address)
        fill_lat(address,j)
    elif(df.Latitude.isnull().any() and j==38):
        address=toronto_df['Borough'][j]
        ##print(j,address)
        fill_lat(address,j)

21 Forest Hill North & West


/Users/athenawong/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


22 High Park
23 North Toronto West
24 The Annex
25 Parkdale
26 Davisville
27 University of Toronto
28 Runnymede
29 Moore Park
30 Kensington Market
31 Summerhill West
32 CN Tower
33 Rosedale
34 Stn A PO Boxes
35 St. James Town
36 First Canadian Place
37 Church and Wellesley


In [17]:
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.664490,-79.393020
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.811530,-79.195520
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.721840,-79.343400
3,M5C,Downtown Toronto,St. James Town,43.768200,-79.217610
4,M4E,East Toronto,The Beaches,43.649700,-79.382580
5,M5E,Downtown Toronto,Berczy Park,43.665090,-79.438710
6,M5G,Downtown Toronto,Central Bay Street,43.739030,-79.467320
7,M6G,Downtown Toronto,Christie,43.683750,-79.355280
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.713810,-79.488310
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.759480,-79.557250


In [18]:
address='toronto'
geolocator =  Nominatim(user_agent='Toronto_agent')
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print('Latitude of {} is {}'.format(address,lat))
print('Longitude of {} is {}'.format(address,long))

Latitude of toronto is 43.6534817
Longitude of toronto is -79.3839347


In [19]:
tmap = folium.Map(location=[lat,long],zoom_start=10,zoom_control=10)

for lat,long,borr,neigh in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighbourhood']):
    labels='{} {}'.format(borr,neigh)
    label = folium.Popup(labels)
    folium.CircleMarker(
        location=[lat,long],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(tmap)

In [20]:
tmap

In [21]:
CLIENT_ID = '2TY0SUFFEHHC2P2K4HOU3POSOUE2RJPOFK531WUMOWSPBY2X' # your Foursquare ID
CLIENT_SECRET = 'Z0GSSOXDJ3SXUCT5CXC3IM3GC2IVBELN1SF0IV5STAUK11L2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2TY0SUFFEHHC2P2K4HOU3POSOUE2RJPOFK531WUMOWSPBY2X
CLIENT_SECRET:Z0GSSOXDJ3SXUCT5CXC3IM3GC2IVBELN1SF0IV5STAUK11L2


In [22]:
toronto_df['Neighbourhood'].loc[0]

'Regent Park, Harbourfront'

In [23]:
location = geolocator.geocode('Regent Park')
neighbourhood_latitude=location.latitude
neighbourhood_longitude=location.longitude
print(neighbourhood_latitude,neighbourhood_longitude)

43.6607056 -79.3604569


In [24]:
Limit=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    Limit)

In [25]:
results = requests.get(url).json()
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5092fe87e4b0476c6e7375e2',
   'name': 'Regent Park Aquatic Centre',
   'location': {'address': '640 Dundas St. East',
    'crossStreet': 'at Sumach St.',
    'lat': 43.6606003130796,
    'lng': -79.36139162915349,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.6606003130796,
      'lng': -79.36139162915349}],
    'distance': 76,
    'postalCode': 'M5A 2B8',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['640 Dundas St. East (at Sumach St.)',
     'Toronto ON M5A 2B8',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d15e941735',
     'name': 'Pool',
     'pluralName': 'Pools',
     'shortName': 'Pool',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_',
      'suffix': '.png'},
     'primary': True}],
   'p

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
nearby_venues

/Users/athenawong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
2,Sumach Espresso,Coffee Shop,43.658135,-79.359515
3,Thai To Go,Thai Restaurant,43.663418,-79.360710
4,Paintbox Bistro,Restaurant,43.660050,-79.362855
5,Qi sushi,Sushi Restaurant,43.662552,-79.364258
6,Sukhothai,Thai Restaurant,43.658444,-79.365681
7,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
8,Toronto Humane Society,Animal Shelter,43.657698,-79.356583
9,Regent Park,Park,43.660383,-79.361810


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                 
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [48]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,16,16,16,16,16,16
"Brockton, Parkdale Village, Exhibition Place",1,1,1,1,1,1
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,4,4,4,4,4,4
Christie,7,7,7,7,7,7
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [49]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.66449,-79.39302,Queen's Park,43.663946,-79.392180,Park
1,"Regent Park, Harbourfront",43.66449,-79.39302,Philosopher's Walk,43.666894,-79.395597,Park
2,"Regent Park, Harbourfront",43.66449,-79.39302,Hart House Theatre,43.663571,-79.394616,Theater
3,"Regent Park, Harbourfront",43.66449,-79.39302,Koerner Hall,43.667983,-79.395962,Concert Hall
4,"Regent Park, Harbourfront",43.66449,-79.39302,Royal Ontario Museum,43.668367,-79.394813,Museum


In [51]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue']],prefix="",prefix_sep="")
toronto_onehot['Neighbourhood']=toronto_venues['Neighbourhood']

toronto_onehot.head()

,401 Games,7-Eleven,900 Mount Pleasant - Residents Gym,A & C Games,A Good Read,A Nerd's World,Absolute Bakery & Café,Adelaide Club Toronto,Aka Teppan,Albertsons,Ali Baba's Middle Eastern Cuisine,Aliacci Pizza and Pasta,Almond Butterfly,Another Story Bookshop,Apple Eaton Centre,Aris Grill,Aris Place,Aroma Espresso Bar,Art Gallery of Ontario,Artscape Youngplace,Asahi Sushi,Assembly Chef's Hall,Athletic Centre,Aura,Avant Gout,Aztec’s Mine,B Espresso Bar,BMV Books,BYOB Cocktail Emporium,Bad Boy Furniture - Distribution Centre,Bakka Phoenix Books,Balsamico,Balzac's Coffee,Bang Bang Ice Cream & Bakery,Banh Mi Nguyen Huong,Bar Centrale by Terroni,Bar Mercurio,Bar Volo,Barbara Hall Park,Barque Butcher Bar,Barque Smokehouse,Baskin-Robbins,Baton Rouge,Bavarian Bier Cafe,Bazara,Bell Trinity Square,Bellwoods Brewery,Bellwoods Brewery Bottle Shop,Best Buy,Best Western Roehampton Hotel & Suites,Big Trouble Pizza,Bikkuri Japanese Restaurant,Birchmount Community Centre,Birchmount Stadium,Black Oak Cafe,Blackbird Baking Co,Bosk at Shangri-La,Boxcar Social,Boxcar Social Temperance,Brick Street Bakery,Bull & Firkin,Bulldog On The Block,Burrito Bandidos,Burrito Boyz,Bus Stop 202128,Butler's Pantry,Butter Chicken Factory,By the Way Cafe,Byblos Toronto,Bymark,CF Toronto Eaton Centre,CN Tower,COBS Bread,Cabbagetown Brew,Cactus Club Cafe,Cafe Cancan,Cafe Landwer,Cafe Neon,Caffe Doria,Café Pamenar,Café Polonez,Canadian Opera Company,Canoe,Carlos' House of Spice,Centennial Health Club,Central YMCA,Charles Sauriol Parkette,Charlie's Gallery,Chatime,Chatime 日出茶太,Cider House,Cinema Paris,Cineplex Cinemas,Club Monaco,Coach House Printing,Coach House Restaurant,Coffee Island,Cold Tea,Como En Casa,Copacabana Grilled Brazilian,Cranberries,Crepe Delicious,Crown Princess Fine Dining 伯爵名宴,Crêpes à GoGo,Curry's Art Store Ltd.,DAVIDsTEA,Dairy Queen,DanceLifeX Centre,Dangerous Lagoon,Dark Horse Espresso Bar,David A. Balfour Park,De La Mer,Delta Hotels by Marriott Toronto,Delta Toronto Club Lounge,Delta Variety,Densgrove Park,Design Exchange,Dineen @CommerceCourt,Dineen Coffee,Dolcetto Bar & Grill,Dollarama,Domani Restaurant & Wine Bar,Domino's Pizza,Don Mills Rd. & Eglinton Ave E,DoubleTree by Hilton,Dovercourt Park,Downtown Toronto,Dumpling House,EQ Village Food Markets,Echo Sushi,Ed's Real Scoop,Eggsmart,Eggspectation Bell Trinity Square,El Almacen Yerba Mate Cafe + Gallery,El Rey,El Trompo,Empress Walk,Enterprise Rent-A-Car,Equinox Bay Street,Essence of Life Organics,Estiatorio Volos,Evviva Restaurant,Extra Butter,F'Amelia,FIKA Cafe,FILM CAFE,FLOCK Rotisserie + Greens,FUEL+,Fabarnak,Famoso Neapolitan Pizzeria,Fionn MacCool's Yonge & Davisville,Fitness Two 4,Five Doors North,Flaming Stove,Four Seasons Centre for the Performing Arts,Foxley Bistro,Fratelli Fresh,Fresh on Bloor,FreshCo,Friendly Stranger - Cannabis Culture Shop,Future Bistro,Fuwa Fuwa Japanese Pancakes,Gabby's,Galleria Smokers Choice,Gardiner Museum,Gelato Simply Italian,Gift Shop,Glad Day Bookshop,Global Cheese,Global Pet Food,Golden Patty,GoodLife Fitness North York Don Mills and Eglinton,GoodLife Fitness Toronto Bell Trinity Centre,Grano,Green Air Mechanical,Green Grotto 綠的小窩,Greenhouse Juice Co,Greg's Ice Cream,Grey Gardens,Gym,Gyubee,Hakim Optical,Handy Variety,Hanji,Happy Bakery & Pastries,Hart House Theatre,Harvest Wagon,Harvey's,Hasty Market,Heritage Family Pantry,Hero Certified Burgers,Hibiscus,High Park Swimming Pool,High Park Village,Hilton,Ho's Team Barber Shop,Homeway Restaurant & Brunch,Hooked,Hordern Pavilion,Hudson's Bay,Hy's Steakhouse,IMAX,Icha Tea 一茶間,Il Paesano Pizzeria & Restaurant,Il Sogno,Independent City Market,Indigo,Innis Cafe,Innis Town Hall,Inter Steer,Isabel Bader Theatre,Isabella Hotel,JaBistro,Jack in the Box,Japango,Jean Sibelius Square,Jimmy's Coffee,John & Sons Oyster House,Juicy Dumpling,Kensington Food Market,Ki Modern Japanese + Bar,Kid Icarus,Kimpton St. George,King Taps,Kingsway Boxing Club,Kitchen Food Fair,Koerner Hall,Konjiki Ramen,Korean Grill House,Kothur I

In [54]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,401 Games,7-Eleven,900 Mount Pleasant - Residents Gym,A & C Games,A Good Read,A Nerd's World,Absolute Bakery & Café,Adelaide Club Toronto,Aka Teppan,Albertsons,Ali Baba's Middle Eastern Cuisine,Aliacci Pizza and Pasta,Almond Butterfly,Another Story Bookshop,Apple Eaton Centre,Aris Grill,Aris Place,Aroma Espresso Bar,Art Gallery of Ontario,Artscape Youngplace,Asahi Sushi,Assembly Chef's Hall,Athletic Centre,Aura,Avant Gout,Aztec’s Mine,B Espresso Bar,BMV Books,BYOB Cocktail Emporium,Bad Boy Furniture - Distribution Centre,Bakka Phoenix Books,Balsamico,Balzac's Coffee,Bang Bang Ice Cream & Bakery,Banh Mi Nguyen Huong,Bar Centrale by Terroni,Bar Mercurio,Bar Volo,Barbara Hall Park,Barque Butcher Bar,Barque Smokehouse,Baskin-Robbins,Baton Rouge,Bavarian Bier Cafe,Bazara,Bell Trinity Square,Bellwoods Brewery,Bellwoods Brewery Bottle Shop,Best Buy,Best Western Roehampton Hotel & Suites,Big Trouble Pizza,Bikkuri Japanese Restaurant,Birchmount Community Centre,Birchmount Stadium,Black Oak Cafe,Blackbird Baking Co,Bosk at Shangri-La,Boxcar Social,Boxcar Social Temperance,Brick Street Bakery,Bull & Firkin,Bulldog On The Block,Burrito Bandidos,Burrito Boyz,Bus Stop 202128,Butler's Pantry,Butter Chicken Factory,By the Way Cafe,Byblos Toronto,Bymark,CF Toronto Eaton Centre,CN Tower,COBS Bread,Cabbagetown Brew,Cactus Club Cafe,Cafe Cancan,Cafe Landwer,Cafe Neon,Caffe Doria,Café Pamenar,Café Polonez,Canadian Opera Company,Canoe,Carlos' House of Spice,Centennial Health Club,Central YMCA,Charles Sauriol Parkette,Charlie's Gallery,Chatime,Chatime 日出茶太,Cider House,Cinema Paris,Cineplex Cinemas,Club Monaco,Coach House Printing,Coach House Restaurant,Coffee Island,Cold Tea,Como En Casa,Copacabana Grilled Brazilian,Cranberries,Crepe Delicious,Crown Princess Fine Dining 伯爵名宴,Crêpes à GoGo,Curry's Art Store Ltd.,DAVIDsTEA,Dairy Queen,DanceLifeX Centre,Dangerous Lagoon,Dark Horse Espresso Bar,David A. Balfour Park,De La Mer,Delta Hotels by Marriott Toronto,Delta Toronto Club Lounge,Delta Variety,Densgrove Park,Design Exchange,Dineen @CommerceCourt,Dineen Coffee,Dolcetto Bar & Grill,Dollarama,Domani Restaurant & Wine Bar,Domino's Pizza,Don Mills Rd. & Eglinton Ave E,DoubleTree by Hilton,Dovercourt Park,Downtown Toronto,Dumpling House,EQ Village Food Markets,Echo Sushi,Ed's Real Scoop,Eggsmart,Eggspectation Bell Trinity Square,El Almacen Yerba Mate Cafe + Gallery,El Rey,El Trompo,Empress Walk,Enterprise Rent-A-Car,Equinox Bay Street,Essence of Life Organics,Estiatorio Volos,Evviva Restaurant,Extra Butter,F'Amelia,FIKA Cafe,FILM CAFE,FLOCK Rotisserie + Greens,FUEL+,Fabarnak,Famoso Neapolitan Pizzeria,Fionn MacCool's Yonge & Davisville,Fitness Two 4,Five Doors North,Flaming Stove,Four Seasons Centre for the Performing Arts,Foxley Bistro,Fratelli Fresh,Fresh on Bloor,FreshCo,Friendly Stranger - Cannabis Culture Shop,Future Bistro,Fuwa Fuwa Japanese Pancakes,Gabby's,Galleria Smokers Choice,Gardiner Museum,Gelato Simply Italian,Gift Shop,Glad Day Bookshop,Global Cheese,Global Pet Food,Golden Patty,GoodLife Fitness North York Don Mills and Eglinton,GoodLife Fitness Toronto Bell Trinity Centre,Grano,Green Air Mechanical,Green Grotto 綠的小窩,Greenhouse Juice Co,Greg's Ice Cream,Grey Gardens,Gym,Gyubee,Hakim Optical,Handy Variety,Hanji,Happy Bakery & Pastries,Hart House Theatre,Harvest Wagon,Harvey's,Hasty Market,Heritage Family Pantry,Hero Certified Burgers,Hibiscus,High Park Swimming Pool,High Park Village,Hilton,Ho's Team Barber Shop,Homeway Restaurant & Brunch,Hooked,Hordern Pavilion,Hudson's Bay,Hy's Steakhouse,IMAX,Icha Tea 一茶間,Il Paesano Pizzeria & Restaurant,Il Sogno,Independent City Market,Indigo,Innis Cafe,Innis Town Hall,Inter Steer,Isabel Bader Theatre,Isabella Hotel,JaBistro,Jack in the Box,Japango,Jean Sibelius Square,Jimmy's Coffee,John & Sons Oyster House,Juicy Dumpling,Kensington Food Market,Ki Modern Japanese + Bar,Kid Icarus,Kimpton St. George,King Taps,Kingsway Boxing Club,Kitchen Food Fair,Koerner Hall,Konjiki Ramen,Korean Grill 

In [56]:
num_top_venues = 5
for neigh in toronto_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                        venue  freq
0        TTC Bus #29 Dufferin  0.06
1                      Rexall  0.06
2  North Of Brooklyn Pizzeria  0.06
3                     FreshCo  0.06
4     Happy Bakery & Pastries  0.06


----Brockton, Parkdale Village, Exhibition Place----
                         venue  freq
0                  Suydam Park   1.0
1                    401 Games   0.0
2                    Rosalinda   0.0
3  Ripley's Aquarium of Canada   0.0
4         River Tai Restaurant   0.0


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                                          venue  freq
0                                   Red Lobster  0.03
1                                       Japango  0.03
2                                         Roots  0.03
3                                     Poke Guys  0.03
4  GoodLife Fitness Toronto Bell Trinity Centre  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, 

               venue  freq
0     little bits hq  0.25
1  Kitchen Food Fair  0.25
2       Tim Horton's  0.25
3    Tournament Park  0.25
4          401 Games  0.00


----Toronto Dominion Centre, Design Exchange----
                                    venue  freq
0                 Summerhill Market North  0.12
1      900 Mount Pleasant - Residents Gym  0.12
2                                 Winners  0.12
3                     TTC Bus #11 Bayview  0.12
4  Best Western Roehampton Hotel & Suites  0.12


----University of Toronto, Harbord----
                              venue  freq
0  Red Fish Blue Fish Creative Cafe  0.03
1               Bakka Phoenix Books  0.03
2                      Queen's Park  0.03
3                   Athletic Centre  0.03
4               The Dessert Kitchen  0.03


